In [1]:
# import modules
import mechanicalsoup
import pandas as pd
import ethnicolr as ec
import re
from pathlib import Path

# test queries
test = pd.concat(map(pd.read_csv, [str(Path().resolve().parents[2]) + "\Reference\Indian-Female-Names.csv",
                                   str(Path().resolve().parents[2]) + "\Reference\Indian-Male-Names.csv",
                                   str(Path().resolve().parents[2]) + "\Reference\Indian-Surnames.csv"]))
test = test.dropna()
test['first name'] = test['first name'].map(lambda x: x if x == " " else x.split()[0])
test.reset_index(drop=True)
test = test.drop(test[test['first name'].map(lambda x: any(check in x for check in [",", ".", "@", "-", "`", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]))].index)
test.reset_index(drop=True)
test = test.drop(test[test['first name'].map(lambda x: False if x == " " else not bool(re.match('[a-zA-Z]', x)))].index)
test = test.drop_duplicates().reset_index(drop=True)

# dictionary to hold data
data = {
    'First Name' : [],
    'Last Name' : [],
    'Email' : []
}

# open browser
browser = mechanicalsoup.StatefulBrowser()

# iterate through all queries
for i in test.index:
    # make next query
    f = test.iat[i, 0]
    if f == " ":
        f = "+"
    l = test.iat[i, 1]
    if l == " ":
        l = "+"
    q = f + l
    
    # go to directory
    browser.open('https://directory.dal.ca/?q=' + q)
    
    print("QUERY: %s" %q)

    # parse raw code
    soup= browser.get_current_page()
    
    results = soup.find_all("table")

    if results != []:
        if len(results[-1].find("tr").findAll("th")) != 4:
            print("\tNo Results Found.")
            continue
        for prof in results[-1].findAll("tr")[1:]:
            cols = prof.findAll("td")
            full_name = cols[0].get_text().split()
            fname = full_name[0]
            lname = full_name[-1]
            email = cols[3].find("a")
            if email == None:
                email = "No Email"
            else:
                email = email.string.strip()
            
            # store contact info in data dictionary
            data['First Name'] += [fname]
            data['Last Name'] += [lname]
            data['Email'] += [email]

            # print contact info
            print("\t"+fname, lname, email)
    else:
        print("\tNo Results Found.")

# close browser
browser.close()

# create pandas dataframe and remove any duplicate entries, add manually retrieved contact info
df = pd.DataFrame(data)
df = df.drop_duplicates().reset_index(drop=True)
    
# determine ethnicity
df = ec.pred_wiki_name(df,list(df)[1],list(df)[0])
    
# print dataframe and write data to csv file
display(df)
df.to_csv("dalu parsed.csv")

Using TensorFlow backend.


QUERY: shivani+
	No Results Found.
QUERY: isha+
	Trisha Aylward Trisha.Aylward@Dal.Ca
	Alisha Johnson Alisha.Johnson@Dal.Ca
	Keisha Turner Keisha.Turner@dal.ca
QUERY: shyani+
	No Results Found.
QUERY: divya+
	No Results Found.
QUERY: mansi+
	No Results Found.
QUERY: mazida+
	No Results Found.
QUERY: pooja+
	No Results Found.
QUERY: kajal+
	No Results Found.
QUERY: meena+
	No Results Found.
QUERY: sonam+
	No Results Found.
QUERY: buity+
	No Results Found.
QUERY: hina+
	Sundari Pashupathinathan sundari.pashupathi@Dal.Ca
QUERY: shakshi+
	No Results Found.
QUERY: anita+
	Juanita Haas J.Haas@Dal.Ca
	Anita MacDonald Anita.MacDonald@Dal.Ca
	Anita Mountain abillard@dal.ca
QUERY: neetu+
	No Results Found.
QUERY: anshu+
	No Results Found.
QUERY: kanika+
	No Results Found.
QUERY: manju+
	Manjula Devaraj MDevaraj@dal.ca
QUERY: reena+
	No Results Found.
QUERY: neha+
	No Results Found.
QUERY: khushboo+
	No Results Found.
QUERY: aasmin+
	No Results Found.
QUERY: jyoti+
	No Results Found.
QUERY: riya+

,First Name,Last Name,Email,race,"Asian,GreaterEastAsian,EastAsian","Asian,GreaterEastAsian,Japanese","Asian,IndianSubContinent","GreaterAfrican,Africans","GreaterAfrican,Muslim","GreaterEuropean,British","GreaterEuropean,EastEuropean","GreaterEuropean,Jewish","GreaterEuropean,WestEuropean,French","GreaterEuropean,WestEuropean,Germanic","GreaterEuropean,WestEuropean,Hispanic","GreaterEuropean,WestEuropean,Italian","GreaterEuropean,WestEuropean,Nordic"
0,Trisha,Aylward,Trisha.Aylward@Dal.Ca,"GreaterEuropean,British",0.001942,0.009740,0.003625,0.006870,0.005944,0.861908,0.001169,0.045671,0.017424,0.002126,0.025428,0.014630,0.003522
1,Alisha,Johnson,Alisha.Johnson@Dal.Ca,"GreaterEuropean,British",0.001220,0.003395,0.013010,0.014878,0.018916,0.801278,0.006551,0.091085,0.016997,0.003146,0.016480,0.006494,0.006550
2,Keisha,Turner,Keisha.Turner@dal.ca,"GreaterEuropean,British",0.002471,0.010907,0.013474,0.011051,0.014929,0.858835,0.001160,0.039500,0.021341,0.001656,0.018153,0.005082,0.001440
3,Sundari,Pashupathinathan,sundari.pashupathi@Dal.Ca,"Asian,IndianSubContinent",0.000304,0.000017,0.996272,0.000238,0.000642,0.002160,0.000016,0.000021,0.000013,0.000026,0.000273,0.000014,0.000004
4,Juanita,Haas,J.Haas@Dal.Ca,"GreaterEuropean,WestEuropean,Germanic",0.000442,0.002348,0.005312,0.006712,0.025518,0.031953,0.063959,0.142856,0.045942,0.468708,0.132558,0.046984,0.026708
5,Anita,MacDonald,Anita.MacDonald@Dal.Ca,"GreaterEuropean,British",0.003741,0.008131,0.009678,0.003291,0.002824,0.898085,0.003659,0.019343,0.010984,0.003421,0.021325,0.010246,0.005274
6,Anita,Mountain,abillard@dal.ca,"GreaterEuropean,British",0.014285,0.015155,0.100843,0.020690,0.041433,0.478360,0.034083,0.071650,0.104058,0.009336,0.067899,0.029721,0.012490
7,Manjula,Devaraj,MDevaraj@dal.ca,"Asian,IndianSubContinent",0.000171,0.000023,0.996532,0.000696,0.001359,0.000473,0.000033,0.000037,0.000066,0.000033,0.000547,0.000024,0.000008
8,Brittany,Bergstrom,Brittany.Bergstrom@Dal.Ca,"GreaterEuropean,Jewish",0.003522,0.006695,0.002125,0.004913,0.003973,0.178970,0.045086,0.398094,0.033323,0.079676,0.116858,0.115108,0.011657
9,Tiffany,Coolen-Jewers,Tiffany.Coolen-Jewers@Dal.Ca,"GreaterEuropean,British",0.002251,0.001740,0.002722,0.003372,0.002176,0.932306,0.000196,0.009079,0.013477,0.001587,0.013221,0.017153,0.000719
